In [2]:
import os
from pyspark.sql import SparkSession
import nbimporter
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, TimestampType
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import lag 
import datetime
import time
import requests


spark = SparkSession\
        .builder \
        .appName("pyspark-notebook") \
        .master("spark://spark-master:7077") \
        .config("spark.executor.memory", "1g") \
        .config("hive.metastore.uris", "thrift://hive:9083") \
        .config("spark.sql.warehouse.dir","/opt/hive/data/warehouse") \
        .enableHiveSupport() \
        .getOrCreate()



    

24/01/11 14:41:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from Functions.functions import read_table, create_hive_external_table
def calculating_volatility_and_market_cap(spark):
    
    df = read_table(spark, "bronze", "crypto_data")
    
    windowSpec  = Window.partitionBy("currency").orderBy("currency")
    
    df = df.withColumn("priceUsd_lagged",lag("priceUsd",1).over(windowSpec))
    df = df.withColumn("market_cap", F.col("priceUsd") * F.col("circulatingsupply"))
    df = df.withColumn("volatility", (F.col("priceUsd") - F.col("priceUsd_lagged")) / F.col("priceUsd_lagged"))
    df = df.withColumn("volatility_percentage", (F.col("volatility") * 100))
    df = df.select(F.col("date"), F.col("time"), F.col("priceusd").cast("double"), F.col("circulatingsupply").cast("double"), F.col("priceusd_lagged").cast("double"), F.col("volatility"), F.col("volatility_percentage"),F.col("market_cap"), F.col("currency"))
    
    create_hive_external_table(spark , df, "silver", "crypto_data", "currency")

In [4]:
calculating_volatility_and_market_cap(spark)

24/01/11 14:42:23 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [5]:
spark.stop()